In [1]:
#| hide
from sherlock.core import *
from fastcore.utils import first

# Sherlock Domains Python SDK

> AI agent & SDK to buy and managed domains from [Sherlock Domains](https://www.sherlockdomains.com/)

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Fewsats/sherlock-python.git
```

```sh
$ pip install sherlock-domains
```


[repo]: https://github.com/Fewsats/sherlock-python
[pypi]: https://pypi.org/project/sherlock-domains/


## How to use

Create a Sherlock instance with a private key for the agent to use. If no key is provided, a new one will be generated and saved to the default config file.

In [2]:
s = Sherlock()
s

Sherlock(pubkey=90ba884688884277e49080712f386eebc88806efa8345ca937f75fe80950156d)

You can search for a domain and request to purchase it. Purchasing a domain requires contact information as mandated by ICANN.

In [3]:
sr = s.search("trakwiska.com")
sr

{'id': '7f53df53-7090-4c3f-a41a-55bd345a8f95',
 'created_at': '2025-01-10T08:22:09.172Z',
 'available': [{'name': 'trakwiska.com',
   'tld': 'com',
   'tags': [],
   'price': 1105,
   'currency': 'USD',
   'available': True},
  {'name': 'trakwiska.net',
   'tld': 'net',
   'tags': [],
   'price': 1185,
   'currency': 'USD',
   'available': True},
  {'name': 'trakwiska.org',
   'tld': 'org',
   'tags': [],
   'price': 939,
   'currency': 'USD',
   'available': True},
  {'name': 'trakwiska.store',
   'tld': 'store',
   'tags': [],
   'price': 488,
   'currency': 'USD',
   'available': True},
  {'name': 'trakwiska.app',
   'tld': 'app',
   'tags': [],
   'price': 1689,
   'currency': 'USD',
   'available': True},
  {'name': 'trakwiska.dev',
   'tld': 'dev',
   'tags': [],
   'price': 1489,
   'currency': 'USD',
   'available': True},
  {'name': 'trakwiska.io',
   'tld': 'io',
   'tags': [],
   'price': 3489,
   'currency': 'USD',
   'available': True},
  {'name': 'trakwiska.xyz',
   'tld'

Contact information is needed for registering a new domain with the ICANN.

In [4]:
c = Contact(**{
    "first_name": "Test",
    "last_name": "User",
    "email": "test@example.com",
    "address": "123 Test St",
    "city": "Test City",
    "state": "CA",
    "country": "US",
    "postal_code": "12345",
})

s.set_contact_information(
    cfn=c.first_name,
    cln=c.last_name,
    cem=c.email,
    cadd=c.address,
    cct=c.city,
    cst=c.state,
    cpc=c.postal_code,
    ccn=c.country
)


{'message': 'Contact information updated successfully'}

Finalizing a pruchase involves a payment. Sherlock Domains currently supports two payment methods: Credit Card (`credit_card`) and the Lightning Netowrk (`lightning`).
By default the credit card method is used. The method will then return a checkout URL that can be used to complete the payment. In the case of lightning the response will contain a lightning invoice.

In [5]:
s.purchase_domain(sr['id'], "trakwiska.com")

{'payment_method': {'checkout_url': 'https://checkout.stripe.com/c/pay/cs_live_a1GGYxOdMQW3ML018fHyqpP0QdFPoK5yuy3KgredBSwPk93idtlY5vTqQS#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl',
  'lightning_invoice': None},
 'expires_at': '2025-01-10T08:52:10.708Z'}

You can now use the checkout URL to complete the purchase and the domain will be registered to your agent.

You can view the domains you own with:

In [6]:
s.domains()

[{'id': 'd9b2cc30-c15d-44b9-9d39-5d33da504484',
  'domain_name': 'h402.org',
  'created_at': '2024-12-28T18:58:49.899Z',
  'expires_at': '2024-12-31T18:58:42Z',
  'auto_renew': False,
  'locked': True,
  'private': True,
  'nameservers': [],
  'status': 'active'}]

Below is a list of all the tools that the client offers to manage the domains and purchases.

In [7]:
s.as_tools().map(lambda t: t.__name__)

(#10) ['_me','_set_contact_information','_get_contact_information','_search','_purchase_domain','_domains','_dns_records','_create_dns_record','_update_dns_record','_delete_dns_record']

## AI agents

We will show how to enable your AI assistant to handle payments using [Claudette](https://claudette.answer.ai), Answer.ai convenient wrapper for Claude. You'll need to export your `ANTHROPIC_API_KEY`.

In [8]:
from claudette import Chat, models

In [9]:
model = models[1]; model

'claude-3-5-sonnet-20240620'

Create a Sherlock instance with a public & private key for the agent to use.

Sherlock supports returning all the tools with `s.as_tools()`.

In order to request a purchase, we need to provide a contact information. You can either do so during the class initialization or manually. Then we just pass the extra tool.


In [10]:
sp = 'You are a helpful assistant that has access to a domain purchase API.'
chat = Chat(model, sp=sp, tools=s.as_tools())

pr = f"Search if domain 'the-favourite-game.com' is available? If it is request a purchase and process the payment using credit card method."
r = chat.toolloop(pr, trace_func=print)
r

Message(id='msg_01RGrzPSYkPGoof9qAgt3sVB', content=[TextBlock(text="Certainly! I'll search for the domain 'the-favourite-game.com' and if it's available, I'll proceed with the purchase request using the credit card payment method. Let's start with the search.", type='text'), ToolUseBlock(id='toolu_01HY1brwGzrYiCEK2xWrFhBF', input={'q': 'the-favourite-game.com'}, name='_search', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 3003; Out: 104; Cache create: 0; Cache read: 0; Total: 3107)
Message(id='msg_01VqNeG8C9eG4prq6G7ZcLun', content=[TextBlock(text="Great news! The domain 'the-favourite-game.com' is available for purchase. The search results show that it's priced at 1105 cents (USD), which is equivalent to $11.05.\n\nNow that we've confirmed its availability, let's proceed with the purchase request using the credit card payment method. However, before we can do that, we need to make sure th

Excellent! The purchase request for 'the-favourite-game.com' has been processed successfully. Here's what you need to know:

1. The domain 'the-favourite-game.com' is available and has been reserved for your purchase.
2. The payment method selected is credit card, as requested.
3. To complete the purchase, you need to use the provided checkout URL:
   https://checkout.stripe.com/c/pay/cs_live_a1w7dHEJsQeXWtL48V27bJqUA9isb3HWCa43srMdsTJKDZg1ZDzqL6FsZc#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl

4. The payment reservation expires at: 2025-01-10T08:52:20.253Z (about 30 minutes from now)

To complete the purchase, please follow these steps:
1. Click on the checkout URL provided above.
2. You will be redirected to a secure Stripe payment page.
3. Enter your credit card details and complete the payment process.
4. Once the payment is successful, the domain will be registered to you.

Make sure to complete the payment before the expiration time to secure your domain. If you have any issues with the payment process or need any further assistance, please let me know.

<details>

- id: `msg_01KDKMFFZ5ELDToH3wMbhjps`
- content: `[{'text': "Excellent! The purchase request for 'the-favourite-game.com' has been processed successfully. Here's what you need to know:\n\n1. The domain 'the-favourite-game.com' is available and has been reserved for your purchase.\n2. The payment method selected is credit card, as requested.\n3. To complete the purchase, you need to use the provided checkout URL:\n   https://checkout.stripe.com/c/pay/cs_live_a1w7dHEJsQeXWtL48V27bJqUA9isb3HWCa43srMdsTJKDZg1ZDzqL6FsZc#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl\n\n4. The payment reservation expires at: 2025-01-10T08:52:20.253Z (about 30 minutes from now)\n\nTo complete the purchase, please follow these steps:\n1. Click on the checkout URL provided above.\n2. You will be redirected to a secure Stripe payment page.\n3. Enter your credit card details and complete the payment process.\n4. Once the payment is successful, the domain will be registered to you.\n\nMake sure to complete the payment before the expiration time to secure your domain. If you have any issues with the payment process or need any further assistance, please let me know.", 'type': 'text'}]`
- model: `claude-3-5-sonnet-20240620`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 4495, 'output_tokens': 532}`

</details>